In [2]:
!pip install -Uqq ipdb
# for debugging/ to use stoppers type: ipdb.set_trace()
import ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.1 MB/s eta 0:00:00


In [3]:
!pip install dbnomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.2 MB/s eta 0:00:00


In [4]:
import dbnomics
import pandas as pd
import numpy as np
import time

In [5]:
!git clone https://git.nomics.world/dbnomics/dbnomics-python-client.git
%cd dbnomics-python-client
!pip install --editable .

Cloning into 'dbnomics-python-client'...
remote: Enumerating objects: 701, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 701 (delta 4), reused 0 (delta 0), pack-reused 679
Receiving objects: 100% (701/701), 2.45 MiB | 2.37 MiB/s, done.
Resolving deltas: 100% (364/364), done.
/content/dbnomics-python-client
Obtaining file:///content/dbnomics-python-client
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for dbnomics (pyproject.toml) ... done
  Created wheel for dbnomics: filename=dbnomics-1.2.5-0.editable-py3-none-any.whl size=26347 sha256=928368e23c40d8a2bc0c9205973a3415478c02079d3dfc7e62d6ed4e1cb23dcc
  Stored in directory: /tmp/pip-ephem-wheel-cache-mwf4qsqm/wheels/60/9e/cf/4826526f197a77f0af2305614ecb51e563ef10bb6d90a22f07
Successfully bui

In [6]:
from dbnomics import fetch_series, fetch_series_by_api_link

In [7]:
provider='IMF'
dataset='FM'
freq='A'
#country_codes=['XR29','XR23','DZ','AO','AR','AU','AT','AZ','BD','BY','BE','BJ','BO','BR','BF','KH','CM','CA','TD','CL','CN','CO','CG','CD','CI','HR','CY','CZ','DK','DO','EC','EG','XS2','XR24','1C_EMEUR','1C_035','1C_EMMENAP','1C_EMFLAT','EE','ET','U2','FI','FR','R22','DE','GH','GR','GN','HT','HN','HK','HU','IS','IN','ID','IR','IE','IL','IT','JP','KZ','KE','KR','KW','KG','LA','LV','LY','LT','1C_230','1C_229','1C_231','1C_235','1C_234','1C_232','LU','MG','R23','MY','ML','MT','MX','MD','MN','MA','MZ','MM','NP','NL','NZ','NI','NE','NG','NO','1C_Oil_producers','OM','PK','PG','PE','PH','PL','PT','QA','RO','RU','RW','SA','SN','SG','SK','SI','ZA','ES','LK','SD','SE','CH','TJ','TZ','TH','TR','UG','UA','AE','GB','US','UY','UZ','VE','VN','YE','ZM','ZW']
#country_codes=['AR','BR','CA','CL','HK','CN','EG','EE','FR','DE','IN','ID','IE','IL','IT','JP','MY','MX','NL','NG','PH','RU','SG','SK','SI','ZA','ES','CH','TW','TH','TR','GB','US',]
country_codes=['AR','BR','CA','CL']
variables=['GGCB_G01_PGDP_PT','GGXWDN_G01_GDP_PT','GGXCNL_G01_GDP_PT','GGXONLB_G01_GDP_PT'] #,'GGCBP_G01_PGDP_PT','G_X_G01_GDP_PT','G_XWDG_G01_GDP_PT','GGXWDN_G01_GDP_PT','GGXCNL_G01_GDP_PT','GGXONLB_G01_GDP_PT','GGR_G01_GDP_PT']

In [8]:
def fetch_data(provider, dataset, freq, max_nb_series=1352):
    return dbnomics.fetch_series(provider, dataset, freq, max_nb_series=max_nb_series)

In [9]:
def filter_data(data, country_codes, variables):

    #filterting the data based on country code, variables, and year
    filtered_data = data[
        (data['REF_AREA'].isin(country_codes)) &
        (data['INDICATOR'].isin(variables)) &
        (data['original_period'].astype(int) >= 1970)
    ]
    return filtered_data

In [10]:
def restructure_data(df, year=1970):

   #try:
        # extracting the dates as 4 numbers
        df.loc[:, 'year'] = df['original_period'].apply(lambda x: x[:4]).astype('int')
        df.loc[:, 'original_period'] = df['original_period'].apply(lambda x: int(x[:4]))
        #df = df[df['year'] >= year]  (already filtered)

        #reformatting the series code
        df.loc[:, 'series_code'] = df['series_code'].apply(lambda x: '.'.join(x.split('.')[1:]))
        df.loc[:, 'description'] = df['Indicator'] + ', ' + df['Reference Area'] + ', ' + df['Frequency']

        #calculate the max year and reset the index
        max_period = pd.DataFrame(df.groupby(['series_code'])['year'].agg('max'))
        max_period.reset_index(inplace=True)

        # merging the max year to 'original_period'
        df = pd.merge(df, max_period, how='left', left_on='series_code', right_on='series_code', suffixes=('', '_max'))

        # renaming column names
        df = df.loc[:, ['series_code', 'description', 'original_period', 'year_max', 'value']]
        df.columns = ['code', 'description', 'period', 'cutoff', 'value']
        return df

    # for debugging
    #except Exception as e:
       # print(f"Error: {e}")
       # return df



In [14]:
def pivot_table(df):

    df['period'] = df['period'].astype(str)

    pivot_df = df.set_index(['code', 'description', 'cutoff', 'period'])['value'].unstack('period')

    # reset index
    pivot_df.reset_index(inplace=True)

    return pivot_df


In [15]:
def save_csv(path,df):
  df.to_csv(path,index=False)

In [16]:
#batch operation to fetch all of the dataset
data = fetch_data(provider, dataset, freq)
# filtering the data
filtered_data = filter_data(data, country_codes, variables)
# restructuring the data
restructured_data = restructure_data(filtered_data)

# structuring the data into a pivot table
pivot_data = pivot_table(restructured_data)
# saving the data to colab vm
save_csv('path.csv', pivot_data)
# display the pivot table data
pivot_data

<ipython-input-10-09348c61fbd2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'year'] = df['original_period'].apply(lambda x: x[:4]).astype('int')
<ipython-input-10-09348c61fbd2>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:, 'original_period'] = df['original_period'].apply(lambda x: int(x[:4]))
<ipython-input-10-09348c61fbd2>:6: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain 

period,code,description,cutoff,1991,1992,1993,1994,1995,1996,1997,...,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028
0,AR.GGCB_G01_PGDP_PT,Cyclically adjusted balance (% of potential GD...,2028,NaN,NaN,NaN,-1.674259,-1.094153,-2.373138,-2.611358,...,-3.429391,-4.985978,-3.272749,-3.846680,-2.310663,-2.308543,-0.970533,0.044862,0.260648,0.239998
1,AR.GGXCNL_G01_GDP_PT,Net lending/borrowing (also referred as overal...,2028,NaN,NaN,NaN,-1.249802,-2.074606,-2.810610,-1.837621,...,-4.395052,-8.588337,-4.321065,-3.848642,-3.985362,-3.726958,-1.894408,-0.519894,-0.087966,0.063340
2,AR.GGXONLB_G01_GDP_PT,Primary net lending/borrowing (also referred a...,2028,NaN,NaN,NaN,0.147192,-0.408225,-0.965789,0.252194,...,-0.402003,-6.178561,-2.519124,-1.804628,-1.616285,-0.531116,0.420304,1.389797,1.974774,2.037790
3,BR.GGCB_G01_PGDP_PT,Cyclically adjusted balance (% of potential GD...,2028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.368014,-10.168681,-2.249914,-3.176021,-7.660248,-6.242283,-5.434126,-4.862911,-4.472168,-4.375652
4,BR.GGXCNL_G01_GDP_PT,Net lending/borrowing (also referred as overal...,2028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.031861,-11.869568,-2.545212,-3.104854,-7.117943,-6.025670,-5.336011,-4.823373,-4.444622,-4.359962
5,BR.GGXONLB_G01_GDP_PT,Primary net lending/borrowing (also referred a...,2028,NaN,NaN,NaN,NaN,NaN,NaN,-0.586905,...,-0.335398,-7.858900,1.985767,2.145163,-1.241523,-0.192137,0.239355,0.727726,1.087102,1.086379
6,BR.GGXWDN_G01_GDP_PT,"Net debt (% of GDP), Brazil, Annual",2028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,54.698838,61.369929,55.816089,57.063399,60.670553,63.709224,66.183728,68.014025,69.421549,70.836487
7,CA.GGCB_G01_PGDP_PT,Cyclically adjusted balance (% of potential GD...,2028,-7.607372,-8.066107,-8.053530,-7.079740,-5.582781,-2.438377,0.314835,...,-0.162498,-9.203877,-3.733682,-1.128078,-0.726064,-0.465986,-0.523893,-0.463661,-0.309603,-0.153818
8,CA.GGXCNL_G01_GDP_PT,Net lending/borrowing (also referred as overal...,2028,-8.359398,-9.199149,-8.934765,-6.935978,-5.489640,-3.052566,0.038702,...,-0.017289,-10.910398,-4.381384,-0.813686,-0.727454,-0.637622,-0.510976,-0.447300,-0.306679,-0.153297
9,CA.GGXONLB_G01_GDP_PT,Primary net lending/borrowing (also referred a...,2028,-3.166409,-3.973492,-3.726054,-1.866859,0.084654,2.140646,4.719900,...,0.103606,-10.469249,-4.966294,-1.250752,-0.969597,-0.704084,-0.509874,-0.346640,-0.164874,0.011663


In [ ]:
from google.colab import files # to download the file to local machine
files.download('path.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>